In [127]:
# Import libraries
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import numpy as np
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import warnings
import random
import csv
warnings.filterwarnings("ignore")

In [128]:
# Import csv file
cities_path = "../data/hotels_items.csv"

In [129]:
# Create dataframe
cities_df = pd.read_csv(cities_path, usecols = ["city"])

In [130]:
cities_df.head()

,city
0,"Mutterstadt, Germany"
1,"Passau, Germany"
2,"Hamburg, Germany"
3,"Stralsund, Germany"
4,"Wolfsburg, Germany"


In [131]:
# Sanity check
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727491 entries, 0 to 727490
Data columns (total 1 columns):
city    727491 non-null object
dtypes: object(1)
memory usage: 5.6+ MB


In [132]:
cities_df.drop_duplicates(inplace = True)

In [133]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27194 entries, 0 to 727408
Data columns (total 1 columns):
city    27194 non-null object
dtypes: object(1)
memory usage: 424.9+ KB


In [134]:
city_list = cities_df["city"].tolist()

In [135]:
# # Create empty columns for the values that will be pulled through API
hotels_df = pd.DataFrame()
hotels_df["rating"] = np.nan
hotels_df["name"] = None
hotels_df["address"] = None
hotels_df["locality"] = None
hotels_df["landmarks"] = None
hotels_df["price"] = np.nan

In [136]:
# List lenght
len(city_list)

27194

In [ ]:
options = Options()
options.headless = True
driver = webdriver.Chrome('/usr/bin/chromedriver', chrome_options=options)
url = "https://www.hotels.com/"

#response.implicitly_wait(10)

with open(r'../data/hotels_com_scrape.csv', 'a') as f:
    csv_writer = csv.writer(f)
    for city in list(city_list):
        #print('Processing: ' + city)

        searchKey = city
        checkInDate = '08/27/2020' #Format %m/%d/%Y
        checkOutDate = '08/29/2020' #Format %m/%d/%Y

        driver.get(url)

        searchKeyElement = driver.find_elements_by_name("q-destination")
        checkInElement = driver.find_elements_by_name("q-localised-check-in")
        checkOutElement = driver.find_elements_by_name("q-localised-check-out")
        submitButton = driver.find_elements_by_css_selector("button.cta-strong")
        closeIconForPopup = driver.find_elements_by_css_selector("span.close-icon")
        closeTypeahead = driver.find_elements_by_css_selector("button.cta-link")
        closeTypeahead = driver.find_elements_by_css_selector("button.cta-link")

        if searchKeyElement and checkInElement and checkOutElement and closeIconForPopup:
            #print('All search elements found.')
            searchKeyElement[0].clear()
            searchKeyElement[0].send_keys(searchKey)
            checkInElement[0].clear()
            checkInElement[0].send_keys(checkInDate)
            checkOutElement[0].clear()
            checkOutElement[0].send_keys(checkOutDate)
            closeIconForPopup[0].click()
            
            # Try to close the the autocomplete box
            searchKeyElement[0].send_keys(Keys.ARROW_DOWN)
            time.sleep(1)
            searchKeyElement[0].send_keys(Keys.ENTER)
            
            time.sleep(2)
            #submitButton[0].click()
            #print('Pressed the submit button.')

            #print('Scrolling a few times.')
            for i in range(3):
                driver.find_element_by_tag_name('body').send_keys(Keys.END)
                time.sleep(0.5)

            hotels = driver.find_elements_by_css_selector('section.hotel-wrap')

            for index, hotel in enumerate(hotels):
                hotel_name = hotel.find_element_by_css_selector('a.property-name-link').text.strip()
                try:
                    hotel_rating = float(hotel.find_element_by_css_selector('div.ta-logo').text.split()[-1])
                except:
                    hotel_rating = "na"
                hotel_address = hotel.find_element_by_css_selector('span.address').text
                hotel_locality = hotel.find_element_by_css_selector("a.map-link").text.strip()
                hotel_landmarks = hotel.find_element_by_css_selector("ul.property-landmarks").text
                try:
                    hotel_price = float(hotel.find_element_by_css_selector("a.price-link").find_element_by_tag_name("ins").text.strip("$"))
                except:
                    hotel_price = "na"

                #print("Saving...")
                row = [city, hotel_name, hotel_rating, hotel_address, hotel_locality, hotel_landmarks, hotel_price]
                print(row)
                csv_writer.writerow(row)
        #print("Done!")
        city_list.remove(city)
        print("\n" + str(len(city_list)) + " cities remaining. \n")

driver.quit()
print("We done.")

['Can Picafort, Spain', 'Grupotel Gran Vista & SPA', 4.5, 'Ctra. Arta - Pto Alcudia, Can Picafort, Santa Margalida, Mallorca, 07458, Spain', 'Can Picafort', '0.7 miles to Es Figueral de Son Real\n0.7 miles to Finca Pública de Son Real', 'na']
['Can Picafort, Spain', 'Apartamentos YourHouse Acapulco', 4.0, 'Calle José Trias, 22, Can Picafort, Santa Margalida, Mallorca, 07458, Spain', 'Can Picafort', '0.7 miles to Es Figueral de Son Real\n0.7 miles to Finca Pública de Son Real', 'na']
['Can Picafort, Spain', 'BQ Can Picafort Hotel', 4.5, 'Carrer Arenal, 24, Can Picafort, Santa Margalida, Mallorca, 07458, Spain', 'Can Picafort', '0.6 miles to Es Figueral de Son Real\n0.6 miles to Finca Pública de Son Real', 'na']
['Can Picafort, Spain', 'EIX Platja Daurada', 4.0, 'Passeig Colom, Can Picafort, Santa Margalida, Mallorca, 07458, Spain', 'Can Picafort', '1.1 miles to Es Figueral de Son Real\n1.1 miles to Finca Pública de Son Real', 'na']
['Can Picafort, Spain', 'Zafiro Can Picafort', 4.5, "C/

In [123]:
driver.quit()